## 🛳️ Titanic — SVM

Projeto de classificação supervisionada para previsão de sobrevivência no Titanic utilizando XGBoost, com foco em modelos baseados em boosting, avaliação estatística rigorosa e seleção objetiva do melhor modelo.

### Destaques

* Pipeline completo com pré-processamento versionado e integração via `Pipeline`.
* Comparação entre Baseline, Random Search, Refine e Otimização Bayesiana.
* Avaliação com validação cruzada 10-fold, ROC-AUC e acurácia.
* Otimização automática de threshold, além do limiar padrão.
* Testes estatísticos (t-test pareado) para validação dos ganhos.
* Relatório técnico automatizado com ranking, vencedor e análise de generalização.


In [4]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from pathlib import Path
import time

from scipy.stats import ttest_rel

# sklearn
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score

from sklearn.base import BaseEstimator, TransformerMixin, clone

# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic
from src.plot_metrica_class import *

In [3]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.2.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv").reset_index(drop=True)
X_test  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR / "y_test_raw.csv")


# # =====================================================
# #  🤖 3.Definição dos Modelos
# # =====================================================
model_SVM0 = SVC(kernel =  'linear', random_state = 42,C=1)
pipe_SVM0   = pipe_models(model_RF0,PP2)

In [ ]:
# Baseline
s0 = cross_val_score(pipe_SVM0 , X_train, y_train, cv=10,scoring='roc_auc')

# 2. Testa a performance 
pipe_SVM0.fit(X_train, y_train)
# 3.Otimização de Threshold
best_t_svm0, score_svm0 = best_threshold(pipe_SVM0, X_test, y_test)

y_pred=pipe_SVM0.predict(X_test)
print(f"{'='*70}")
print(f"🎯 Random Forest (Baseline) | cvscores : {s0.mean():.4f} ± {s0.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_test, y_pred))
cm=confusion_matrix(y_test, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")